### Loading the json data

In [5]:
import pandas as pd
import json

#Loading the json files
def load_json_file(file_path): 
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = [json.loads(line) for line in file]  
        print(f"Success: JSON Loaded from {file_path}")
        return data
    except json.JSONDecodeError as e:
        print(f"Failed: Error loading JSON from {file_path}: {e}")  # Handle JSON formatting errors
        return None
    except FileNotFoundError:
        print(f"File not found: {file_path}")   # Handle missing files
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}") # Catch any other unexpected errors
        return None
    
# Load JSON files into Python lists
users_data = load_json_file("users.json")
brands_data = load_json_file("brands.json")
receipts_data = load_json_file("receipts.json")

Success: JSON Loaded from users.json
Success: JSON Loaded from brands.json
Success: JSON Loaded from receipts.json


### Read and normalize the json data

In [8]:
import pandas as pd

def load_and_normalize_json(file_path, explode_column=None, sep='_', n_preview=5):
    try:
        #read and normalize the data 
        df = pd.read_json(file_path, lines=True)            
        df = pd.json_normalize(df.to_dict(orient='records'), sep=sep) 
        
        # Explode nested column if specified
        if explode_column:
            df = df.explode(explode_column)
            df = pd.json_normalize(df[explode_column].dropna(), sep=sep)

        # Preview the data
        print(f"\nPreview of {file_path}:")
        print(df.head(n_preview))
        
        return df
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error
    
# Load and normalize data from JSON files
users_df = load_and_normalize_json('users.json')
brands_df = load_and_normalize_json('brands.json')
receipts_df = load_and_normalize_json('receipts.json', explode_column='rewardsReceiptItemList')



Preview of users.json:
   active      role signUpSource state                  _id_$oid  \
0    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
1    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
2    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
3    True  consumer        Email    WI  5ff1e1eacfcf6c399c274ae6   
4    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   

   createdDate_$date  lastLogin_$date  lastLogin  
0      1609687444800     1.609688e+12        NaN  
1      1609687444800     1.609688e+12        NaN  
2      1609687444800     1.609688e+12        NaN  
3      1609687530554     1.609688e+12        NaN  
4      1609687444800     1.609688e+12        NaN  

Preview of brands.json:
        barcode        category      categoryCode                       name  \
0  511111019862          Baking            BAKING  test brand @1612366101024   
1  511111519928       Beverages         BEVERAGES                  Starbucks

### Convert json to csv 

In [17]:
import json
import csv
from datetime import datetime

# Load MongoDB JSON data and read each line as separate JSON objects
def load_mongo_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = []
        for line in file:
            data.append(json.loads(line))
        return data

# Write data to CSV
def write_to_csv(file_name, data, fieldnames):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()  
        writer.writerows(data)  

# Convert MongoDB data to a structured format
def segregate_data(users_file, brands_file, receipts_file):
    users_data = load_mongo_json(users_file)
    brands_data = load_mongo_json(brands_file)
    receipts_data = load_mongo_json(receipts_file)

    # Extract Users data (Flatten MongoDB document)
    usersnew = []
    for user in users_data:
        usersnew.append({
            # MongoDB ObjectID ($oid) converted to string
           "user_id": user["_id"]["$oid"],  
            "active": user.get("active", None),
            "created_date": convert_mongo_date(user.get("createdDate", None)),
            "last_login": convert_mongo_date(user.get("lastLogin", None)),
            "role": user.get("role", None),
            "signup_source": user.get("signUpSource", None),
            "state": user.get("state", None)
        })
    
    # Define the fieldnames for Users CSV
    users_fieldnames = ["user_id", "active","created_date", "last_login", "role", "signup_source", "state"]
    write_to_csv("users_data.csv", usersnew, users_fieldnames)

    # Extract Brands and CPGs data
    brandsnew = []
    cpgs = {}
    for brand in brands_data:
        brandsnew.append({
            "brand_id": brand["_id"]["$oid"],
            "barcode": brand.get("barcode", None),
            "brand_code": brand.get("brandCode", None),
            "category": brand.get("category", None),
            "category_code": brand.get("categoryCode", None),
            "name": brand.get("name", None),
            "top_brand": brand.get("topBrand", None),
            "cpg_id": brand.get("cpg", {}).get("$id", {}).get("$oid", None) 
        })
        
        cpgs[brand.get("cpg", {}).get("$id", {}).get("$oid", None)] = {             
            "cpg_id": brand.get("cpg", {}).get("$id", {}).get("$oid", None), 
            "name": brand.get("cpg", {}).get("$ref", None)
        }

    # Define the fieldnames for Brands CSV
    brands_fieldnames = ["brand_id", "barcode", "brand_code", "category", "category_code", "name", "top_brand", "cpg_id"]
    write_to_csv("brands_data.csv", brandsnew, brands_fieldnames)
    
    # Write CPGs data to CSV
    cpgs_list = list(cpgs.values())
    cpgs_fieldnames = ["cpg_id", "name"]
    write_to_csv("cpgs_data.csv", cpgs_list, cpgs_fieldnames)
    
    # Extract Receipts and ReceiptItems data
    receiptsnew  = []
    receipt_items = []
    for receipt in receipts_data:
        receiptsnew.append({
            # MongoDB ObjectID ($oid) converted to string
            "r_id": receipt["_id"]["$oid"], 
            "bonus_points_earned": receipt.get("bonusPointsEarned", None),
            "bonus_points_earned_reason": receipt.get("bonusPointsEarnedReason", None),
            "create_date": convert_mongo_date(receipt.get("createDate", None)),
            "date_scanned": convert_mongo_date(receipt.get("dateScanned", None)),
            "finished_date": convert_mongo_date(receipt.get("finishedDate", None)),
            "modify_date": convert_mongo_date(receipt.get("modifyDate", None)),
            "points_awarded_date": convert_mongo_date(receipt.get("pointsAwardedDate", None)),
            "points_earned": receipt.get("pointsEarned", None),
            "purchase_date": convert_mongo_date(receipt.get("purchaseDate", None)),
            "purchased_item_count": receipt.get("purchasedItemCount", None),
            "rewards_receipt_status": receipt.get("rewardsReceiptStatus", None),
            "total_spent": receipt.get("totalSpent", None),
            "user_id": receipt.get("userId", None)
        })
       
        rewardsReceiptItemList = receipt.get("rewardsReceiptItemList", []) 
        for item in rewardsReceiptItemList:
            receipt_items.append({
                "receipt_id": receipt["_id"]["$oid"],  
                "barcode": item.get("barcode", None),
                "description": item.get("description", None),
                "final_price": item.get("finalPrice", None),
                "item_price": item.get("itemPrice", None),
                "quantity_purchased": item.get("quantityPurchased", None),
                "needs_fetch_review": item.get("needsFetchReview", None),
                "needs_fetch_review_reason": item.get("needsFetchReviewReason", None),
                "partner_item_id": item.get("partnerItemId", None),
                "prevent_target_gap_points": item.get("preventTargetGapPoints", None),
                "user_flagged_barcode": item.get("userFlaggedBarcode", None),
                "user_flagged_description": item.get("userFlaggedDescription", None),
                "user_flagged_new_item": item.get("userFlaggedNewItem", None),
                "user_flagged_price": item.get("userFlaggedPrice", None),
                "user_flagged_quantity": item.get("userFlaggedQuantity", None),
                "points_not_awarded_reason": item.get("pointsNotAwardedReason", None),
                "points_payer_id": item.get("pointsPayerId", None),
                "rewards_group": item.get("rewardsGroup", None),
                "rewards_product_partner_id": item.get("rewardsProductPartnerId", None)
            })

    # Define the fieldnames for Receipts CSV
    receipts_fieldnames = ["r_id", "bonus_points_earned", "bonus_points_earned_reason", "create_date", "date_scanned", 
                           "finished_date", "modify_date", "points_awarded_date", "points_earned", "purchase_date", 
                           "purchased_item_count", "rewards_receipt_status", "total_spent", "user_id"]
    write_to_csv("receipts_data.csv", receiptsnew, receipts_fieldnames)
    
    # Define the fieldnames for ReceiptItems CSV
    receipt_items_fieldnames = ["receipt_id", "barcode", "description", "final_price", "item_price", 
                                "quantity_purchased", "needs_fetch_review", "needs_fetch_review_reason","partner_item_id", "prevent_target_gap_points", 
                                "user_flagged_barcode", "user_flagged_description", "user_flagged_new_item", 
                                "user_flagged_price", "user_flagged_quantity", "points_not_awarded_reason", "points_payer_id", 
                                "rewards_group", "rewards_product_partner_id"]
    write_to_csv("receipt_items_data.csv", receipt_items, receipt_items_fieldnames)

    print("json to csv completed and Files are saved as per below:")
    print("users_data.csv")
    print("brands_data.csv")
    print("cpgs_data.csv")
    print("receipts_data.csv")
    print("receipt_items_data.csv")

# Function to convert MongoDB date format (timestamps) to SQL-friendly format
def convert_mongo_date(mongo_date):
    if mongo_date:
        if isinstance(mongo_date, dict) and "$date" in mongo_date:
            # Extract the timestamp from the $date field and convert it to datetime
            return datetime.utcfromtimestamp(mongo_date["$date"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
    return None

# Paths to original MongoDB JSON files
users_file = 'users.json'
brands_file = 'brands.json'
receipts_file = 'receipts.json'

# Segregate the data
segregate_data(users_file, brands_file, receipts_file)

json to csv completed and Files are saved as per below:
users_data.csv
brands_data.csv
cpgs_data.csv
receipts_data.csv
receipt_items_data.csv
